In [26]:
import json
from openai import OpenAI
import pandas as pd
from IPython.display import Image, display
import hashlib
import json
import time

client = OpenAI(api_key="sk-proj-sScFMZr4GK3XaQ5mcYFYT3BlbkFJ1orIjm1gKSNT0q30KBMZ")

In [27]:
resumes = pd.read_csv('data/trunc-resumes.csv')
postings = pd.read_csv('data/trunc-postings.csv')
display(postings)
display(resumes)

,description
0,Job descriptionA leading real estate firm in N...
1,"At Aspen Therapy and Wellness , we are committ..."
2,The National Exemplar is accepting application...
3,Senior Associate Attorney - Elder Law / Trusts...
4,Looking for HVAC service tech with experience ...
...,...
95,"QualificationsExperience:Data Entry, 4 years (..."
96,The firm:Dynamic SRG is a boutique political c...
97,"Global Sculpted Streetwear fashion brand, Nake..."
98,"Executive Cleaning Services, Inc. is not a mai..."


,Resume,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR
...,...,...
95,DIRECTOR OF HR Executive Pr...,HR
96,HR SENIOR SPECIALIST Career Ove...,HR
97,HR CUSTOMER SERVICE REPRESENTATIVE ...,HR
98,HR SERVICES REPRESENTATIVE Summ...,HR


In [28]:
# system_prompt="""
# Your goal is to use someone's resume to tell me if they are qualified for a job.
# You will be provided with a resume and a job description. Your output will be a json object containing only one of five categorizations: "not qualified", "minimally qualified", "somewhat qualified", "qualified", "very qualified".
# {
# 	categorization: string // Level of qualification
# } 
# """

# def Get_Qualifications(resume, job_description):
# 	user_prompt = f"Here is my resume:{resume}, and here is the job description: {job_description}"
# 	response = client.chat.completions.create(
# 		model="gpt-3.5-turbo",
# 		temperature=0.1,
# 		response_format={"type":"json_object"},
# 		messages=[
# 			{ "role": "system", "content":system_prompt },
# 			{ "role": "user", "content": user_prompt }
# 		]
# 	)
# 	return response.choices[0].message.content

In [29]:
# for i in range(3):
# 	resume = resumes['Resume'][i]
# 	job_description = postings['description'][i]
# 	result = Get_Qualifications(resume, job_description)
# 	print(result)

In [30]:
# # Creating an array of json tasks
# tasks = []
# for job_description in postings['description']:
# 	for resume_text in resumes['Resume']:
# 		user_prompt = f"Here is my resume:{resume}, and here is the job description: {job_description}"
# 		key = hashlib.md5(f"{resume_text}{job_description}".encode()).hexdigest()

# 		task = {
#         	"custom_id": f"{key}",
#         	"method": "POST",
#         	"url": "/v1/chat/completions",
#         	"body": {
#             	"model": "gpt-3.5-turbo",
#             	"temperature": 0.1,
#             	"response_format": { "type": "json_object" },
#             	"messages": [
#                 	{ "role": "system", "content": system_prompt },
#                 	{ "role": "user", "content": user_prompt }
# 				]
# 			}
# 		}
# 		tasks.append(task)

In [31]:
# # Creating the batch file
# file_name = "data/batch_tasks_labels.jsonl"

# with open(file_name, 'w') as file:
#     for obj in tasks:
#         file.write(json.dumps(obj) + '\n')

In [32]:
# # Uploading the Batch file to OpenAI
# batch_file = client.files.create(
#   file=open(file_name, "rb"),
#   purpose="batch"
# )
# print(batch_file)

In [33]:
# # Creating the batch job:
# batch_job = client.batches.create(
#   input_file_id=batch_file.id,
#   endpoint="/v1/chat/completions",
#   completion_window="24h"
# )

In [34]:
# # Checking the batch status
# batch_job = client.batches.retrieve(batch_job.id)
# print(batch_job)

In [35]:
# # Retrieving results
# result_file_id = batch_job.output_file_id
# result = client.files.content(result_file_id).content

# result_file_name = "data/batch_job_results_labels.jsonl"

# with open(result_file_name, 'wb') as file:
#     file.write(result)
    
# # Loading data from saved file
# results = []
# with open(result_file_name, 'r') as file:
#     for line in file:
#         # Parsing the JSON string into a dict and appending to the list of results
#         json_object = json.loads(line.strip())
#         results.append(json_object)

In [36]:
# # Create a cache from the results
# cache = {}
# for res in results:
# 	key = res['custom_id']
# 	task_id = res['custom_id']
# 	result_str = res['response']['body']['choices'][0]['message']['content']
# 	result_obj = json.loads(result_str)
# 	result = result_obj['categorization']
# 	if result in ("qualified", "very qualified"):
# 		cache[key] = 1
# 	else:
# 		cache[key] = 0
# 	print(f"{task_id},{result}")

# with open('cache.json', 'w') as cache_file:
# 		json.dump(cache, cache_file)

In [40]:
system_prompt="""
Your goal is to use someone's resume to tell me if they are qualified for a job.
You will be provided with a resume and a job description. Your output will be a json object containing only one of five categorizations: "not qualified", "minimally qualified", "somewhat qualified", "qualified", "very qualified".
{
	categorization: string // Level of qualification
} 
"""

def CreateTask(resume: str, job_description: str):
	user_prompt = f"Here is my resume:{resume}, and here is the job description: {job_description}"
	key = hashlib.md5(f"{resume}{job_description}".encode()).hexdigest()
	task = {
        "custom_id": f"{key}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-3.5-turbo",
            "temperature": 0.1,
            "response_format": { "type": "json_object" },
            "messages": [
                { "role": "system", "content": system_prompt },
                { "role": "user", "content": user_prompt }
			]
		}
	}
	return task

def WaitForJobToFinish(batch_job_id):
	"""
	Waits for the batch job to finish by polling every 1 minute.

	Parameters:
	- batch_job_id: The ID of the batch job to wait for.

	Returns:
	- The final status of the batch job.
	"""
	while True:
		batch_job = client.batches.retrieve(batch_job_id)

		# Check the status of the batch job
		status = batch_job.status
		print(f"Current status: {status}")

		if status in ['completed', 'failed']:
			return status
        
		# Wait for the specified interval before polling again
		time.sleep(60)

def TasksToCache(tasks, batch_file_name, result_file_name, cache_file_name):

	# Create Batch file
	with open(batch_file_name, 'w') as file:
		for obj in tasks:
			file.write(json.dumps(obj) + '\n')

	# Upload the Batch file to OpenAI
	batch_file = client.files.create(
		file=open(batch_file_name, "rb"),
		purpose="batch"
	)
	print(batch_file)
	
	# Creating the batch job:
	batch_job = client.batches.create(
		input_file_id=batch_file.id,
		endpoint="/v1/chat/completions",
		completion_window="24h"
	)

	status = WaitForJobToFinish(batch_job.id)

	if (status == 'failed'):
		batch_job = client.batches.retrieve(batch_job.id)
		print('batch job failed!')
		print(batch_job)
		return
	
	# Retrieving results
	batch_job = client.batches.retrieve(batch_job.id)
	result_file_id = batch_job.output_file_id
	result = client.files.content(result_file_id).content

	result_file_name

	with open(result_file_name, 'wb') as file:
		file.write(result)

	# Loading data from saved file
	results = []
	with open(result_file_name, 'r') as file:
		for line in file:
			# Parsing the JSON string into a dict and appending to the list of results
			json_object = json.loads(line.strip())
			results.append(json_object)
	# Create a cache from the results
	cache = {}
	for res in results:
		key = res['custom_id']
		task_id = res['custom_id']
		result_str = res['response']['body']['choices'][0]['message']['content']
		result_obj = json.loads(result_str)
		result = result_obj['categorization']
		if result in ("qualified", "very qualified"):
			cache[key] = 1
		else:
			cache[key] = 0
	print(f"{task_id},{result}")

	with open(cache_file_name, 'w') as cache_file:
		json.dump(cache, cache_file)

def AutoCacheFiller():
	tasks = []
	for job_description in postings['description']:
		for resume_text in resumes['Resume']:
			tasks.append(CreateTask(resume_text, job_description))

	batch_size = 100
	split_tasks = [tasks[i:i+batch_size] for i in range(0, len(tasks), batch_size)]

	# Print the number of batches created
	print(f"Number of batches: {len(split_tasks)}")

	i = 0
	for batch in split_tasks:
		TasksToCache(batch, f"auto_data/{i}_batch.jsonl", f"auto_data/{i}_result.jsonl", f"auto_data/{i}_test_cache.jsonl")
		i+=1

AutoCacheFiller()

Number of batches: 100
FileObject(id='file-2h3NLMweQsCdSAcUbEWkGfsB', bytes=1015089, created_at=1720735759, filename='0_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)
Current status: validating
Current status: completed
d6151306659472373daf19294c76eeb7,not qualified
FileObject(id='file-YmVGvZwtKbSfUTUHvsIqbESZ', bytes=1117689, created_at=1720735823, filename='1_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)
Current status: validating
Current status: completed
1850c9d223a3d534a88abe595237c041,not qualified
FileObject(id='file-0zxOgtbIVL7PWGXLue3Jb7LS', bytes=807589, created_at=1720735886, filename='2_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)
Current status: validating
Current status: in_progress
Current status: completed
92f397a4656badba982ddd826e9d16c4,not qualified
FileObject(id='file-dJnYqrfpqv1QwQSEdgJxkCOz', bytes=921589, created_at=1720736010, filename='3_ba